In [1]:
import pandas as pd
import numpy as np

from pandas.api.types import is_string_dtype

pd.options.display.max_columns = None

In [2]:
port_clientes = pd.read_csv("../datasets/portfolio_clientes.csv", sep=",")
port_comunicados = pd.read_csv("../datasets/portfolio_comunicados.csv", sep=",")
port_geral = pd.read_csv("../datasets/portfolio_geral.csv", sep=",", nrows=200000, skiprows=range(1, 8000000, 40))
port_tpv = pd.read_csv("../datasets/portfolio_tpv.csv", sep=",")

In [33]:
def tratar_dataset(dataset):

    for coluna in dataset.columns:
        if is_string_dtype(dataset[coluna]):
            dataset[coluna] = dataset[coluna].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            dataset[coluna] = dataset[coluna].str.upper()
            
    dataset.columns = map(str.upper, dataset.columns)
    
    return dataset


def converter_para_datetime(dataset, lst_colunas, formato):
    for coluna in lst_colunas:
        dataset[coluna] = pd.to_datetime(dataset[coluna], format=formato)
    
    
def agrupa_e_segmenta_porcentagem(dataframe, coluna_groupby, coluna_tipo):
    tipos = list(filter(lambda elem: not pd.isna(elem), dataframe[coluna_tipo].unique()))
    
    tipos_dict = {}
    for tipo in tipos:
        query = dataframe.query(f'{coluna_tipo} == "{tipo}"')
        contagem = query.groupby(coluna_groupby).size()
        porcentagem = round(contagem / contagem.sum() * 100, 2)

        tipo_dataframe = pd.DataFrame()
        tipo_dataframe['contagem'] = contagem
        tipo_dataframe['porcentagem'] = porcentagem

        tipos_dict[tipo] = tipo_dataframe.sort_values(by=['contagem'], ascending=False)

    return tipos_dict


In [4]:
lst_datasets = [port_clientes, port_comunicados, port_geral, port_tpv]

for dataset in lst_datasets:
    dataset = tratar_dataset(dataset)
    dataset.dropna(inplace=True)
    dataset.drop_duplicates(inplace=True)


In [5]:
converter_para_datetime(port_tpv, ["DT_TRANSACAO"], "%Y%m%d")
converter_para_datetime(port_geral, ["DT_REF_PORTFOLIO", "SAFRA", "DT_CONTRATO", "DT_DESEMBOLSO", "DT_VENCIMENTO", "DT_WO"], "%Y-%m-%d")
converter_para_datetime(port_comunicados, ["DT_REF_PORTFOLIO", "DATA_ACAO"], "%Y-%m-%d")

In [6]:
port_clientes.drop(port_clientes.index[port_clientes["ESTADO"] == "ND"], inplace = True)
port_clientes.drop(port_clientes.index[port_clientes["SUBSEGMENTO"] == "NONE"], inplace = True)

In [7]:
lst_nr_suspensos = set(port_geral.query("STATUS_CONTRATO == 'SUSPENDED'").NR_DOCUMENTO.tolist())
lst_nr_pagos = set(port_geral.query("STATUS_CONTRATO == 'SETTLED'").NR_DOCUMENTO.tolist())

In [8]:
intersection_set = set.intersection(set(lst_nr_pagos), set(lst_nr_suspensos))

In [9]:
lst_nr_pago_sem_suspenso = lst_nr_pagos - lst_nr_suspensos

# PESSOAS QUE QUITARAM A DIVIDA E NUNCA FORAM SUSPENSAS

In [17]:
lst_nr_suspenso_sem_pagar = lst_nr_suspensos - lst_nr_pagos

# PESSOAS QUE FORAM SUSPENSAS E NUNCA PAGARAM A DIVIDA

In [ ]:
port_clientes, port_comunicados, port_geral, port_tpv

In [79]:
port_comunicados

,CONTRATO_ID,DT_REF_PORTFOLIO,DATA_ACAO,TIPO_ACAO,ACAO,STATUS
0,0D823DDFF1839EB938DCCC68586AE365,2021-01-16,2021-01-18,HSM,CAMPANHAOBSERVACAO,LIDO
1,C2F8A21671DBAEAA4157E03682C639E1,2021-05-22,2021-05-24,HSM,CAMPANHAOBSERVACAO,ENTREGUE
2,E19A222C233D46FF4984489C13AA1B98,2020-09-15,2020-09-15,HSM,CAMPANHAOBSERVACAO,NAO ENTREGUE
3,C24FC7630FE5B89928BA9478C2CDD9B2,2021-07-20,2021-07-20,HSM,CAMPANHAOBSERVACAO,NAO ENTREGUE
4,0ABE9C2F21E941F2BD3B086277E33EBC,2020-12-26,2020-12-28,HSM,CAMPANHAOBSERVACAO,ENTREGUE
...,...,...,...,...,...,...
403735,C7DC25F386144912B90500FFD45E71C5,2020-10-19,2020-10-19,HSM,CAMPANHABOLETOQUITADO,NAO ENTREGUE
403736,76C9917AF4FA0810BD2E5638523A1A30,2020-06-19,2020-06-19,HSM,CAMPANHABOLETOQUITADO,LIDO
403737,724407E5260F93C0E3EA385564BDC731,2021-06-28,2021-06-28,HSM,CAMPANHABOLETOQUITADO,NAO ENTREGUE
403738,42B5D6594F79EE300CE0966C894B04C5,2021-09-26,2021-09-28,HSM,CAMPANHABOLETOQUITADO,NAO ENTREGUE


In [80]:
port_comunicados.groupby(by=["CONTRATO_ID"]).count()

,DT_REF_PORTFOLIO,DATA_ACAO,TIPO_ACAO,ACAO,STATUS
CONTRATO_ID,,,,,
000180509391A5AC66FF83CAE603FFB8,4,4,4,4,4
000C35A61297EDADC2842F6D5B4028E1,24,24,24,24,24
000DCDC93A545EE45A1AEE85EF85C34A,70,70,70,70,70
0014CCCD47B66A47AF4E62C4DCBE95F1,22,22,22,22,22
001621E2D725AB3D3773692745BE79B2,18,18,18,18,18
...,...,...,...,...,...
FFF4BF15D5B48A5C1BDA798EA77290D1,2,2,2,2,2
FFF6EDC99168D63A10EA51A7F808FF38,8,8,8,8,8
FFFB0D6EC51D163CA8C69900F748D61B,44,44,44,44,44


In [12]:
contratos_pagos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(lst_nr_pago_sem_suspenso)]


In [85]:
contratos_pagos

,CONTRATO_ID,DT_REF_PORTFOLIO,SAFRA,NR_DOCUMENTO,STATUS_CONTRATO,DT_CONTRATO,DT_DESEMBOLSO,DT_VENCIMENTO,DT_WO,PRAZO,VLR_DESEMBOLSADO,VLR_TARIFA,JUROS_MES,JUROS_DIARIO,PERC_RETENCAO,VLR_PGTO_REALIZADO,VLR_PGTO_ESPERADO,VLR_SALDO_DEVEDOR,VLR_SALDO_DEVEDOR_ESPERADO,DSP,DSPP,FLAG_TRANSACAO
0,1EB0E13F83F336146227CCC9EFB08C47,2022-03-01,2020-07-01,938513461B4A4FCD2BF99FFD8F12B9BE,SETTLED,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,0.08400,0.00,46.87,0.00,0.00,0,0,0
1,8B03675F597974CA96F06648E96AF6D4,2022-03-01,2020-08-01,4303BD71EE3FE80ABA4EFC8DBD6C7D70,SETTLED,2020-08-13,2020-08-17,2022-02-13,2023-02-13,30.78,17100.00,0.0,0.05388,0.00176,0.08076,0.00,45.57,0.00,0.00,0,0,0
7,C48198C89F94CA4FD075BF30B889B47D,2020-10-22,2020-04-01,77846FD227FE5270DBBFE752515D1AF5,ACTIVE,2020-04-15,2020-04-22,2021-07-15,2022-07-15,25.65,1710.00,0.0,0.04788,0.00157,0.05520,30.95,5.03,466.54,1127.39,0,0,0
8,05EDDBA83DC40815A1AA10B6D6259708,2020-10-22,2020-07-01,62290FE7C7682B1F829AE81B1FD08BAF,ACTIVE,2020-07-02,2020-07-06,2021-10-02,2022-10-02,25.65,10514.79,0.0,0.07188,0.00233,0.13200,130.59,34.91,7285.04,8756.04,0,0,0
9,5D441EC4BDEDAA91E530CBFD72AC3966,2022-03-24,2020-07-01,D41D3E7F476557383C0CE4392448F55D,SETTLED,2020-07-21,2020-07-23,2021-10-21,2022-10-21,25.65,23921.19,0.0,0.04788,0.00157,0.10800,0.00,69.75,0.00,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199983,74DC2E4DE6CC77D14BF1AC36529A3A96,2021-11-24,2020-09-01,4E9FB2D22D306C7F94A22B798DA62E82,SETTLED,2020-09-04,2020-09-09,2022-06-04,2023-06-04,35.91,6253.47,0.0,0.05988,0.00195,1.20000,0.00,15.82,0.00,1481.14,0,0,0
199985,3FA4F7070A43186DC41EA2205117D093,2021-11-24,2020-02-01,53DC1E99CB8D17993C09CC3C0F75FBC0,SETTLED,2020-02-18,2020-02-21,2020-11-18,2021-11-18,13.68,6840.00,68.4,0.08388,0.00271,0.25740,0.00,34.23,0.00,0.00,0,0,0
199994,1D7FE0015E269FC1BFFC78E80D8789C7,2021-02-28,2020-08-01,DED4D7A9D876EBF9DFDD656B5AC9AE88,ACTIVE,2020-08-11,2020-08-13,2021-12-11,2022-12-11,27.36,19911.24,0.0,0.05988,0.00195,0.13200,0.00,59.35,14181.97,13534.26,4,11,0
199998,226DEF9CE410C15DDDE927801581BEC0,2021-11-12,2020-08-01,1393488C35CB0BC0C67ED857402B6FF1,SETTLED,2020-08-12,2020-08-14,2021-12-12,2022-12-12,27.36,9442.62,0.0,0.09588,0.00308,1.20000,0.00,34.06,0.00,0.00,0,0,0


In [40]:
contratos_pagos_suspensos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(intersection_set)]

In [21]:
contratos_suspensos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(lst_nr_suspenso_sem_pagar)]

############################################################################


In [13]:
clientes_com_contratos_pago = contratos_pagos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [41]:
contratos_pagos_suspensos_merge = contratos_pagos_suspensos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [22]:
clientes_com_contratos_suspensos = contratos_suspensos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

############################################################################

In [44]:
contratos_pagos_suspensos_merge["TIPO_EMPRESA"].value_counts()

PJ     5604
MEI    3263
PF      851
Name: TIPO_EMPRESA, dtype: int64

In [45]:
clientes_com_contratos_suspensos["TIPO_EMPRESA"].value_counts()

PJ     6902
MEI    3876
PF     1810
Name: TIPO_EMPRESA, dtype: int64

In [70]:
clientes_com_contratos_pago["TIPO_EMPRESA"].value_counts()

PJ     64379
MEI    33311
PF      8659
Name: TIPO_EMPRESA, dtype: int64

In [73]:
clientes_com_contratos_pago.groupby(by=["CONTRATO_ID"]).count()

,DT_REF_PORTFOLIO,SAFRA,NR_DOCUMENTO,STATUS_CONTRATO,DT_CONTRATO,DT_DESEMBOLSO,DT_VENCIMENTO,DT_WO,PRAZO,VLR_DESEMBOLSADO,VLR_TARIFA,JUROS_MES,JUROS_DIARIO,PERC_RETENCAO,VLR_PGTO_REALIZADO,VLR_PGTO_ESPERADO,VLR_SALDO_DEVEDOR,VLR_SALDO_DEVEDOR_ESPERADO,DSP,DSPP,FLAG_TRANSACAO,TIPO_EMPRESA,CIDADE,ESTADO,SUBSEGMENTO,SEGMENTO
CONTRATO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,
000180509391A5AC66FF83CAE603FFB8,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
000C35A61297EDADC2842F6D5B4028E1,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
000DCDC93A545EE45A1AEE85EF85C34A,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
0024224B38EF536444919D635D07FA6E,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
002CA102EE148D919F3BFAE3885CDCFD,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFF6136B712B3B8CF95AA2DDA1FA8EC0,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
FFF6EDC99168D63A10EA51A7F808FF38,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
FFF95C9441DC478A4991F4A2A8C55111,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23


In [64]:
clientes_com_contratos_pago

,CONTRATO_ID,DT_REF_PORTFOLIO,SAFRA,NR_DOCUMENTO,STATUS_CONTRATO,DT_CONTRATO,DT_DESEMBOLSO,DT_VENCIMENTO,DT_WO,PRAZO,VLR_DESEMBOLSADO,VLR_TARIFA,JUROS_MES,JUROS_DIARIO,PERC_RETENCAO,VLR_PGTO_REALIZADO,VLR_PGTO_ESPERADO,VLR_SALDO_DEVEDOR,VLR_SALDO_DEVEDOR_ESPERADO,DSP,DSPP,FLAG_TRANSACAO,TIPO_EMPRESA,CIDADE,ESTADO,SUBSEGMENTO,SEGMENTO
0,1EB0E13F83F336146227CCC9EFB08C47,2022-03-01,2020-07-01,938513461B4A4FCD2BF99FFD8F12B9BE,SETTLED,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,0.08400,0.00,46.87,0.00,0.00,0,0,0,MEI,CAMPO GRANDE,MS,ALIMENTACAO RAPIDA,ALIMENTACAO
1,1EB0E13F83F336146227CCC9EFB08C47,2021-12-12,2020-07-01,938513461B4A4FCD2BF99FFD8F12B9BE,SETTLED,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,0.08400,0.00,46.87,0.00,0.00,0,0,1,MEI,CAMPO GRANDE,MS,ALIMENTACAO RAPIDA,ALIMENTACAO
2,1EB0E13F83F336146227CCC9EFB08C47,2020-11-11,2020-07-01,938513461B4A4FCD2BF99FFD8F12B9BE,ACTIVE,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,0.08400,6.26,46.87,12798.91,13102.71,0,1,1,MEI,CAMPO GRANDE,MS,ALIMENTACAO RAPIDA,ALIMENTACAO
3,1EB0E13F83F336146227CCC9EFB08C47,2020-11-09,2020-07-01,938513461B4A4FCD2BF99FFD8F12B9BE,ACTIVE,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,0.08400,228.75,46.87,12824.86,13162.14,0,0,1,MEI,CAMPO GRANDE,MS,ALIMENTACAO RAPIDA,ALIMENTACAO
4,1EB0E13F83F336146227CCC9EFB08C47,2022-01-01,2020-07-01,938513461B4A4FCD2BF99FFD8F12B9BE,SETTLED,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,0.08400,0.00,46.87,0.00,0.00,0,0,1,MEI,CAMPO GRANDE,MS,ALIMENTACAO RAPIDA,ALIMENTACAO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106344,BFC6D3FED31D03441D5D84A5315C961F,2021-12-18,2021-02-01,656FBE457F792D0E10B27D4A83A869C8,SETTLED,2021-02-19,2021-02-23,2022-05-19,2023-05-19,25.65,6840.00,0.0,0.04788,0.00157,0.07872,0.00,20.35,0.02,1538.36,0,0,1,MEI,OSASCO,SP,ALIMENTACAO RAPIDA,ALIMENTACAO
106345,9F52F197DDBCAB70323E85CAC1EC12CB,2022-04-09,2021-01-01,BC61559E38624C301A3290300FE87CF5,SETTLED,2021-01-14,2021-01-18,2022-04-14,2023-04-14,25.65,35862.12,0.0,0.07188,0.00233,0.03096,0.00,119.86,0.00,0.00,0,0,1,MEI,SAO PAULO,SP,ALIMENTACAO RAPIDA,ALIMENTACAO
106346,45D1187948A519474E33954878933EF5,2022-04-09,2020-08-01,549D3E2A3EE441445B169B320A7D9E5A,SETTLED,2020-08-27,2020-08-31,2021-11-27,2022-11-27,25.65,64224.18,0.0,0.05988,0.00195,0.18000,0.00,200.37,0.00,0.00,0,0,1,MEI,CAMPO BELO,MG,BARES E RESTAURANTES,ALIMENTACAO
106347,E07760294300F3EC6198DA1A0AECDE27,2022-04-06,2021-04-01,56B1D1BC91D6B0BC82B7C98E152970F1,SETTLED,2021-04-05,2021-04-07,2022-07-05,2023-07-05,25.65,4302.36,0.0,0.04788,0.00157,0.16800,0.00,12.72,0.00,0.00,0,0,0,MEI,CIANORTE,PR,OUTROS,BENS DURAVEIS


In [36]:
clientes_com_contratos_pago

,CONTRATO_ID,DT_REF_PORTFOLIO,SAFRA,NR_DOCUMENTO,STATUS_CONTRATO,DT_CONTRATO,DT_DESEMBOLSO,DT_VENCIMENTO,DT_WO,PRAZO,VLR_DESEMBOLSADO,VLR_TARIFA,JUROS_MES,JUROS_DIARIO,PERC_RETENCAO,VLR_PGTO_REALIZADO,VLR_PGTO_ESPERADO,VLR_SALDO_DEVEDOR,VLR_SALDO_DEVEDOR_ESPERADO,DSP,DSPP,FLAG_TRANSACAO,TIPO_EMPRESA,CIDADE,ESTADO,SUBSEGMENTO,SEGMENTO
0,770AC29AC0474F52AC9C7F2704BF8251,2020-11-09,2020-09-01,41A5782BBC753A86D2A5D5C359AE0030,ACTIVE,2020-09-19,2020-09-22,2021-12-19,2022-12-19,25.65,3669.66,0.0,0.07188,0.00233,0.19200,0.39,12.18,3602.83,3409.71,0,6,0,MEI,FORTALEZA,CE,OUTROS,SERVICOS
1,770AC29AC0474F52AC9C7F2704BF8251,2022-04-15,2020-09-01,41A5782BBC753A86D2A5D5C359AE0030,ACTIVE,2020-09-19,2020-09-22,2021-12-19,2022-12-19,25.65,3669.66,0.0,0.07188,0.00233,0.60000,0.00,12.18,1560.39,0.00,190,190,0,MEI,FORTALEZA,CE,OUTROS,SERVICOS
2,770AC29AC0474F52AC9C7F2704BF8251,2021-11-22,2020-09-01,41A5782BBC753A86D2A5D5C359AE0030,SUSPENDED,2020-09-19,2020-09-22,2021-12-19,2022-12-19,25.65,3669.66,0.0,0.07188,0.00233,0.18264,0.00,12.18,1180.22,0.00,46,46,0,MEI,FORTALEZA,CE,OUTROS,SERVICOS
3,770AC29AC0474F52AC9C7F2704BF8251,2020-09-26,2020-09-01,41A5782BBC753A86D2A5D5C359AE0030,ACTIVE,2020-09-19,2020-09-22,2021-12-19,2022-12-19,25.65,3669.66,0.0,0.07188,0.00233,0.19200,0.00,12.18,3672.24,3644.25,1,1,0,MEI,FORTALEZA,CE,OUTROS,SERVICOS
4,770AC29AC0474F52AC9C7F2704BF8251,2021-06-03,2020-09-01,41A5782BBC753A86D2A5D5C359AE0030,ACTIVE,2020-09-19,2020-09-22,2021-12-19,2022-12-19,25.65,3669.66,0.0,0.07188,0.00233,0.18264,0.00,12.18,1539.91,1929.58,7,7,0,MEI,FORTALEZA,CE,OUTROS,SERVICOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12583,F4DA92BDD6E9C0001BDD926139BFE1C1,2021-06-17,2021-02-01,7AAA1B2FB284D3A7AFCF42AE0D961525,ACTIVE,2021-01-30,2021-02-02,2022-05-30,2023-05-30,27.36,6364.62,0.0,0.05988,0.00195,0.10800,4.86,19.29,5746.15,4799.50,0,23,1,MEI,BARUERI,SP,BARES E RESTAURANTES,ALIMENTACAO
12584,C413305A7D32FA57A82E27B616087742,2022-03-01,2020-12-01,884FA5F72BF945C4DCC092059BF7E2B7,SUSPENDED,2020-12-23,2020-12-28,2022-06-23,2023-06-23,30.78,21304.89,0.0,0.03588,0.00118,0.14400,0.00,50.72,10401.88,2205.37,29,29,0,MEI,AVARE,SP,AUTOMOTIVO,BENS DURAVEIS
12585,3A0FB1730F38CA0B3E39E616BCB40789,2022-03-15,2021-05-01,C99498D53FCF31461E9AAE1A46220653,SUSPENDED,2021-05-26,2021-05-28,2022-08-26,2023-08-26,25.65,17891.73,0.0,0.07188,0.00233,0.07200,0.00,60.38,16843.33,657.14,13,206,1,MEI,BELFORD ROXO,RJ,SUPERMERCADOS,SUPERMERCADO/FARMACIA
12586,478A33BDA26A6B95B0C1D8B0275AC0ED,2022-02-03,2021-04-01,FE32D76E7E5505D86453BD21DBA53706,SUSPENDED,2021-04-07,2021-04-09,2022-07-07,2023-07-07,25.65,30780.00,0.0,0.05988,0.00195,0.10800,0.00,97.04,13856.87,3836.75,17,17,1,PJ,MARABA,PA,SAUDE,SERVICOS


Prazo/parcelas do contrato da linha de crédito, geralmente o prazo médio é de 8 meses.

https://www.bndes.gov.br/wps/portal/site/home/financiamento/guia/prazos-periodicidade-pagamento

In [15]:
lst_nr_pago_sem_suspenso = list(lst_nr_pago_sem_suspenso)

Estado atual do empréstimo, podendo ser "Desembolso confirmado (DisbursementConfirmed), Aceito (Accepted), Desembolso solicitado (DisbursementRequested), Ativo (Active) ou Quitado (Settled)".